In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

In [2]:
Epochs = 10
batch_size = 64
learning_rate = 1e-4
l2_regularize = 1e-5
momentum = 0.9

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.STL10(root = './data', download = True, split = 'train', transform = transform)
test_data = datasets.STL10(root = './data', download = True, split = 'test', transform = transform)

train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

 94%|█████████▍| 2.49G/2.64G [01:14<00:03, 42.5MB/s] 

In [ ]:
model1 = models.efficientnet_v2_l(weights = models.EfficientNet_V2_L_Weights.DEFAULT)

In [ ]:
model1

In [ ]:
for params in model1.parameters():
    params.require_grad = False

In [ ]:
num_features = model1.classifier[1].in_features
model1.classifier[1] = nn.Linear(num_features, 10)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=learning_rate, weight_decay=l2_regularize)

In [ ]:
print(model1.classifier)

In [ ]:
for _, labels in train_loader:
    print("Label dtype:", labels.dtype)
    print("Min label:", labels.min().item())
    print("Max label:", labels.max().item())
    break

In [ ]:
train_losses, test_losses = [], []
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model1.to(device)

for Epoch in range(Epochs):
    model1.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model1(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    model1.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model1(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
    test_loss = running_loss /len(test_loader.dataset)
    test_losses.append(test_loss)
        
    print(f'Epoch {Epoch + 1}/{Epochs} - Train loss: {train_loss}, ')